### Preparar ambiente

In [1]:
import enum
print(enum.__file__)  
# standard library location should be something like 
# /usr/local/lib/python3.6/enum.py

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/enum.py


In [2]:
import sys
!{sys.executable} -m pip uninstall -y enum34

In [3]:
!{sys.executable} -m pip install awswrangler

In [4]:
!{sys.executable} -m pip install PyAthena

### Conectar con pyathena

In [5]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

In [6]:
import pandas as pd
import awswrangler as wr
import numpy as np
import calendar
import boto3
s3 = boto3.resource('s3')

In [7]:
import os
import pandas as pd
import zipfile
import shutil
from io import StringIO
import io

### Declaración funciones para Impo-Expo s3

In [8]:
def write_dataframe_to_csv_on_s3(dataframe, bucket, filename):
    """ Write a dataframe to a CSV on S3 """
    # Create buffer
    csv_buffer = StringIO()
    # Write dataframe to buffer
    dataframe.to_csv(csv_buffer, sep=";" , line_terminator='\n')
    # Create S3 object
    s3_resource = boto3.resource("s3")
    # Write buffer to S3 object
    s3_resource.Object(bucket, filename).put(Body=csv_buffer.getvalue())
    print("Writing {} records to {}".format(len(dataframe), filename))
    print("S3")
    print(dataframe.shape)
    

In [9]:
def write_dataframe_to_excel_on_s3(dataframe, bucket, filename):
    """ Write a dataframe to a excel on S3 """
    with io.BytesIO() as output:
        with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
            dataframe.to_excel(writer,index=False)
        data = output.getvalue()

    s3 = boto3.resource('s3')
    s3.Bucket(bucket).put_object(Key=filename, Body=data)
    

In [10]:
def import_dataframe_csv_from_s3(bucket, filename):
    client = boto3.client('s3')
    bucket_name = bucket
    object_key = filename
    csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_string),sep=";")
    return df

In [11]:
def import_dataframe_excel_from_s3(bucket, filename):
    client = boto3.client('s3')
    bucket_name = bucket
    object_key = filename
    obj = client.get_object(Bucket=bucket_name, Key=object_key)
    data = obj['Body'].read()
    df = pd.read_excel(io.BytesIO(data), encoding='utf-8')
    return df

In [12]:
def write_file_to_s3(bucket,file_local,file_s3):
    s3.Bucket(bucket).upload_file(file_local,file_s3)

### Importacion de Delta de Fase 1

In [13]:
fecha_inicio = '02-01-2020'
fecha_final = '04-30-2020'
fechas = pd.date_range(fecha_inicio, fecha_final)

In [14]:
pais = 'BR'
nivel_1 = 'id_provincia'
nivel_4 = 'cd_geocmi'
filtro_variable = "n_obs"
filtro_num = "10"

In [15]:
#Computar serie de usuarios homogenea
filtro_variable2 = 'n_obs_madrugada'
filtro_variable22 = 'n_obs_noche'
filtro_num2 = 4
#consulta athena
tabla_usuarios_distancia =  f'historico_usuarios_regiones_{pais}'   
query_filtro = (f''' SELECT caid,count(*) as count_filtro
                     FROM {tabla_usuarios_distancia} 
                     WHERE (({tabla_usuarios_distancia}.{filtro_variable2} + {tabla_usuarios_distancia}.{filtro_variable22})>= {filtro_num2})
                     GROUP BY caid
                     HAVING count(*)>=30''')
df_filtro = cursor.execute(query_filtro).as_pandas()

In [16]:
start_time = time.time()
for fecha in fechas:
    time_fecha = time.time()
    dia = str(fecha.day)
    mes = str(fecha.month)
    query_fecha = ""
    day = str(fecha.day)
    if len(day) == 1:
        day = f'0{day}'
    codigo_fecha = str(fecha.month) + day
    tabla_usuarios_distancia =  f'historico_usuarios_regiones_{pais}'   
    query_filtro = (f''' SELECT *
                         FROM {tabla_usuarios_distancia} 
                         WHERE ({tabla_usuarios_distancia}.month={mes}
                         AND {tabla_usuarios_distancia}.day={dia} 
                         AND {tabla_usuarios_distancia}.iso_country_code ='{pais}' 
                         AND {tabla_usuarios_distancia}.{filtro_variable} >= {filtro_num})''')
    df = cursor.execute(query_filtro).as_pandas()
    df['MasDe1km'] = df['distancia_recorrida'].apply(lambda x: 1 if x>=1 else 0)
    
    
    ##Filtro mas tenue
    #nivel 1
    df_by_day_1=df.groupby(['month','day',nivel_1]).agg({'MasDe1km':'sum','caid':'count'}).reset_index()
    df_by_day_1=df_by_day_1.rename(columns={'caid':'n_users','month':'Mes','day':'Dia'})
    df_by_day_1['%MasDe1km'] = df_by_day_1['MasDe1km']/df_by_day_1['n_users']*100  
    df_median=df[df['MasDe1km']==1].groupby(['month','day',nivel_1]).agg({'distancia_recorrida':'median'}).reset_index()
    df_median=df_median.rename(columns={'distancia_recorrida':'Mediana1KM','month':'Mes','day':'Dia'})
    df_by_day_1=df_by_day_1.merge(df_median, how='left',on=['Mes','Dia',nivel_1])
    df_by_day_1[nivel_1]=df_by_day_1[nivel_1].astype('int')
    df_by_day_1['Mes']=df_by_day_1['Mes'].astype('int')
    df_by_day_1['Dia']=df_by_day_1['Dia'].astype('int')
    df_by_day_1["Dia"]=df_by_day_1["Dia"].map("{:02}".format)
    df_by_day_1["key4"]=df_by_day_1['Mes'].astype(str)+ df_by_day_1["Dia"].astype(str)
    df_by_day_1['key4']=df_by_day_1['key4'].astype('int')
    df_by_day_1['Dia']=df_by_day_1['Dia'].astype('int')    
    if (nivel_1=="id_provincia"):
        df_by_day_1=df_by_day_1.rename(columns={nivel_1:'adm1'})
    #if (dia=='1' and mes=='2'):
    #    df_final_1=df_by_day_1
    #else:
    #    df_final_1=df_final_1.append(df_by_day_1)
    
    
    table_uc = f'historico_latlong_{pais}_{codigo_fecha}_nivel1'
    wr.s3.to_parquet(  # Storing the data and metadata to Data Lake
    df = df_by_day_1,
    dataset = True,
    table = table_uc,
    database = "graphdata",
    path = f"s3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/Results_niveles/Temp_BR/{table_uc}"
    )
    
    
    #nivel 4   
    df_by_day_4=df.groupby(['month','day',nivel_4]).agg({'MasDe1km':'sum','caid':'count'}).reset_index()
    df_by_day_4=df_by_day_4.rename(columns={'caid':'n_users','month':'Mes','day':'Dia'})
    df_by_day_4['%MasDe1km'] = df_by_day_4['MasDe1km']/df_by_day_4['n_users']*100  
    df_median=df[df['MasDe1km']==1].groupby(['month','day',nivel_4]).agg({'distancia_recorrida':'median'}).reset_index()
    df_median=df_median.rename(columns={'distancia_recorrida':'Mediana1KM','month':'Mes','day':'Dia'})
    df_by_day_4=df_by_day_4.merge(df_median, how='left',on=['Mes','Dia',nivel_4])    
    df_by_day_4[nivel_4]=df_by_day_4[nivel_4].astype('int')
    df_by_day_4['Mes']=df_by_day_4['Mes'].astype('int')
    df_by_day_4['Dia']=df_by_day_4['Dia'].astype('int')
    df_by_day_4["Dia"]=df_by_day_4["Dia"].map("{:02}".format)
    df_by_day_4["key4"]=df_by_day_4['Mes'].astype(str)+ df_by_day_4["Dia"].astype(str)
    df_by_day_4['key4']=df_by_day_4['key4'].astype('int')
    df_by_day_4['Dia']=df_by_day_4['Dia'].astype('int')
    if (nivel_4=="cd_geocmi"):
        df_by_day_4=df_by_day_4.rename(columns={nivel_4:'adm4'})
    #if (dia=='1' and mes=='2'):
    #    df_final_4=df_by_day_4
    #else:
    #    df_final_4=df_final_4.append(df_by_day_4)
        
    table_uc = f'historico_latlong_{pais}_{codigo_fecha}_nivel4'
    wr.s3.to_parquet(  # Storing the data and metadata to Data Lake
    df = df_by_day_4,
    dataset = True,
    table = table_uc,
    database = "graphdata",
    path = f"s3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/Results_niveles/Temp_BR/{table_uc}"
    )
    
    
    ##Filtro severo
    #nivel 1
    df2=df.merge(df_filtro,how='inner',on='caid')    
    df_by_day_11=df2.groupby(['month','day',nivel_1]).agg({'MasDe1km':'sum','caid':'count'}).reset_index()
    df_by_day_11=df_by_day_11.rename(columns={'caid':'n_users','month':'Mes','day':'Dia'})
    df_by_day_11['%MasDe1km'] = df_by_day_11['MasDe1km']/df_by_day_11['n_users']*100  
    df_median=df2[df2['MasDe1km']==1].groupby(['month','day',nivel_1]).agg({'distancia_recorrida':'median'}).reset_index()
    df_median=df_median.rename(columns={'distancia_recorrida':'Mediana1KM','month':'Mes','day':'Dia'})
    df_by_day_11=df_by_day_11.merge(df_median, how='left',on=['Mes','Dia',nivel_1])      
    df_by_day_11[nivel_1]=df_by_day_11[nivel_1].astype('int')
    df_by_day_11['Mes']=df_by_day_11['Mes'].astype('int')
    df_by_day_11['Dia']=df_by_day_11['Dia'].astype('int')
    df_by_day_11["Dia"]=df_by_day_11["Dia"].map("{:02}".format)
    df_by_day_11["key4"]=df_by_day_11['Mes'].astype(str)+ df_by_day_11["Dia"].astype(str)
    df_by_day_11['key4']=df_by_day_11['key4'].astype('int')
    df_by_day_11['Dia']=df_by_day_11['Dia'].astype('int')    
    if (nivel_1=="id_provincia"):
        df_by_day_11=df_by_day_11.rename(columns={nivel_1:'adm1'})
    #if (dia=='1' and mes=='2'):
    #    df_final_fs1=df_by_day_11
    #else:
    #    df_final_fs1=df_final_fs1.append(df_by_day_11)
    
    table_uc = f'historico_latlong_{pais}_{codigo_fecha}_nivel1_fs'
    wr.s3.to_parquet(  # Storing the data and metadata to Data Lake
    df = df_by_day_11,
    dataset = True,
    table = table_uc,
    database = "graphdata",
    path = f"s3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/Results_niveles/Temp_BR/{table_uc}"
    ) 
    
    
    #nivel 4
    df2=df.merge(df_filtro,how='inner',on='caid')
    df_by_day_44=df2.groupby(['month','day',nivel_4]).agg({'MasDe1km':'sum','caid':'count'}).reset_index()
    df_by_day_44=df_by_day_44.rename(columns={'caid':'n_users','month':'Mes','day':'Dia'})
    df_by_day_44['%MasDe1km'] = df_by_day_44['MasDe1km']/df_by_day_44['n_users']*100  
    df_median=df2[df2['MasDe1km']==1].groupby(['month','day',nivel_4]).agg({'distancia_recorrida':'median'}).reset_index()
    df_median=df_median.rename(columns={'distancia_recorrida':'Mediana1KM','month':'Mes','day':'Dia'})
    df_by_day_44=df_by_day_44.merge(df_median, how='left',on=['Mes','Dia',nivel_4])      
    df_by_day_44[nivel_4]=df_by_day_44[nivel_4].astype('int')
    df_by_day_44['Mes']=df_by_day_44['Mes'].astype('int')
    df_by_day_44['Dia']=df_by_day_44['Dia'].astype('int')
    df_by_day_44["Dia"]=df_by_day_44["Dia"].map("{:02}".format)
    df_by_day_44["key4"]=df_by_day_44['Mes'].astype(str)+ df_by_day_44["Dia"].astype(str)
    df_by_day_44['key4']=df_by_day_44['key4'].astype('int')
    df_by_day_44['Dia']=df_by_day_44['Dia'].astype('int')
    if (nivel_4=="cd_geocmi"):
        df_by_day_44=df_by_day_44.rename(columns={nivel_4:'adm4'})
    #if (dia=='1' and mes=='2'):
    #    df_final_fs4=df_by_day_44
    #else:
    #    df_final_fs4=df_final_fs4.append(df_by_day_44)
        
    table_uc = f'historico_latlong_{pais}_{codigo_fecha}_nivel4_fs'
    wr.s3.to_parquet(  # Storing the data and metadata to Data Lake
    df = df_by_day_44,
    dataset = True,
    table = table_uc,
    database = "graphdata",
    path = f"s3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/Results_niveles/Temp_BR/{table_uc}"
    )  
    

    tiempo = (time.time()-time_fecha)
    print(f"{mes}{dia}, {int(tiempo)} seg")

21, 72 seg
22, 67 seg
23, 72 seg
24, 72 seg
25, 74 seg
26, 75 seg
27, 73 seg
28, 72 seg
29, 77 seg
210, 73 seg
211, 70 seg
212, 74 seg
213, 73 seg
214, 70 seg
215, 71 seg
216, 75 seg
217, 79 seg
218, 74 seg
219, 73 seg
220, 71 seg
221, 69 seg
222, 68 seg
223, 56 seg
224, 69 seg
225, 72 seg
226, 67 seg
227, 67 seg
228, 72 seg
229, 67 seg
31, 67 seg
32, 70 seg
33, 78 seg
34, 84 seg
35, 72 seg
36, 73 seg
37, 76 seg
38, 84 seg
39, 68 seg
310, 69 seg
311, 66 seg
312, 68 seg
313, 69 seg
314, 70 seg
315, 70 seg
316, 72 seg
317, 66 seg
318, 63 seg
319, 69 seg
320, 65 seg
321, 67 seg
322, 61 seg
323, 64 seg
324, 70 seg
325, 48 seg
326, 49 seg
327, 45 seg
328, 44 seg
329, 44 seg
330, 46 seg
331, 46 seg
41, 50 seg
42, 50 seg
43, 45 seg
44, 45 seg
45, 43 seg
46, 50 seg
47, 46 seg
48, 46 seg
49, 46 seg
410, 44 seg
411, 43 seg
412, 43 seg
413, 43 seg
414, 45 seg
415, 45 seg
416, 49 seg
417, 48 seg
418, 47 seg
419, 45 seg
420, 48 seg
421, 50 seg
422, 50 seg
423, 56 seg
424, 55 seg
425, 55 seg
426, 52

### Control - Filtro 10p

In [17]:
cod_region=pd.read_excel('cod_BR.xlsx', engine='openpyxl')

#### Creo adm1 de referencia

In [18]:
df_final = []
start_time = time.time()
for i,fecha in enumerate(fechas):
    time_fecha = time.time()
    day = str(fecha.day)
    if len(day) == 1:
        day = f'0{day}'
    codigo_fecha = str(fecha.month) + day
    #print(codigo_fecha)
    tabla_usuarios_fecha = f'historico_latlong_{pais}_{codigo_fecha}_nivel1'
    query = f'''SELECT * FROM {tabla_usuarios_fecha}'''
    #print(f"{codigo_fecha}")
    df2 = cursor.execute(query).as_pandas()
    if (codigo_fecha=='201'):
        df_final=df2
    else:
        df_final=df_final.append(df2)

df_final_1 = df_final
tiempo = (time.time()-time_fecha)/60
print((time.time()-start_time)/60)
df_final_1=df_final_1.rename(columns={'mas_de1km':'MasDe1km','_mas_de1km':'%MasDe1km','mediana1_km':'Mediana1KM','mes':'Mes','dia':'Dia'})


2.5088905970255535


In [19]:
df_final_1.columns

Index(['Mes', 'Dia', 'adm1', 'MasDe1km', 'n_users', '%MasDe1km', 'Mediana1KM',
       'key4'],
      dtype='object')

#### control nivel adm4

In [20]:
df_final = []
start_time = time.time()
for i,fecha in enumerate(fechas):
    time_fecha = time.time()
    day = str(fecha.day)
    if len(day) == 1:
        day = f'0{day}'
    codigo_fecha = str(fecha.month) + day
    #print(codigo_fecha)
    tabla_usuarios_fecha = f'historico_latlong_{pais}_{codigo_fecha}_nivel4'
    query = f'''SELECT * FROM {tabla_usuarios_fecha}'''
    #print(f"{codigo_fecha}")
    df2 = cursor.execute(query).as_pandas()
    if (codigo_fecha=='201'):
        df_final=df2
    else:
        df_final=df_final.append(df2)
df_final=df_final.rename(columns={'mas_de1km':'MasDe1km','_mas_de1km':'%MasDe1km','mediana1_km':'Mediana1KM','mes':'Mes','dia':'Dia'})
df_final_4 = df_final
tiempo = (time.time()-time_fecha)/60
print((time.time()-start_time)/60)

3.301035459836324


In [21]:
## control nivel adm4
base_control=df_final_1
base_control_v4=df_final_4
base_control_v4["adm1"]=""
base_control_v4.loc[base_control_v4["adm4"].astype(str).str.len()==7,["adm1"]]=base_control_v4["adm4"][base_control_v4["adm4"].astype(str).str.len()==7].map(lambda x: str(x)[0:2])
base_control4=base_control_v4.groupby(['Mes','Dia','adm1']).agg({'MasDe1km':'sum','n_users':'sum','%MasDe1km':'mean','Mediana1KM':'median'}).reset_index()
base_control4['adm1']=base_control4['adm1'].astype('int')
base_control4['adm1_v1']=""

for ids in list(cod_region['id_ibge'].astype('int')):
    base_control4.loc[base_control4['adm1']== ids , ['adm1_v1']] =  int(cod_region.loc[cod_region['id_ibge']== ids, 'id_provincia'])

base_control4 =base_control4.loc[:,['Mes','Dia','adm1_v1','MasDe1km','n_users','%MasDe1km','Mediana1KM']]
base_control4 =base_control4.rename(columns={'adm1_v1':'adm1','MasDe1km':'MasDe1km_adm4','n_users':'n_users_adm4','%MasDe1km':'%MasDe1km_adm4','Mediana1KM':'Mediana1KM_adm4'})

base_contro4=base_control.merge(base_control4,how='left',on=['Mes','Dia','adm1'])
base_contro4['%MasDe1km_adm4']=base_contro4['MasDe1km_adm4']/base_contro4['n_users_adm4']*100

cora=base_contro4.groupby('adm1')[['MasDe1km','MasDe1km_adm4']].corr().unstack().iloc[:,1].min()
corb=base_contro4.groupby('adm1')[['n_users','n_users_adm4']].corr().unstack().iloc[:,1].min()
corc=base_contro4.groupby('adm1')[['%MasDe1km','%MasDe1km_adm4']].corr().unstack().iloc[:,1].min()

if (min(cora,corb,corc)<0.9): print("OJO") 
else: print(min(cora,corb,corc), "CONTINUE")

0.9998978465736418 CONTINUE


#### Guardar archivos

In [22]:
nivel_4 = 'amd4'
ruta=f'graphdata-mobility-public/Results_niveles/{pais}/almenos10p/{pais}_{nivel_4}_10p.csv'
write_dataframe_to_csv_on_s3(df_final_4, 'iadbprod-csd-hub-analyticaldata', ruta)

Writing 500585 records to graphdata-mobility-public/Results_niveles/BR/almenos10p/BR_amd4_10p.csv
S3
(500585, 9)


### Filtro Base Usuarios

In [23]:
df_final = []
start_time = time.time()
for i,fecha in enumerate(fechas):
    time_fecha = time.time()
    day = str(fecha.day)
    if len(day) == 1:
        day = f'0{day}'
    codigo_fecha = str(fecha.month) + day
    #print(codigo_fecha)
    tabla_usuarios_fecha = f'historico_latlong_{pais}_{codigo_fecha}_nivel1_fs'
    query = f'''SELECT * FROM {tabla_usuarios_fecha}'''
    #print(f"{codigo_fecha}")
    df2 = cursor.execute(query).as_pandas()
    if (codigo_fecha=='201'):
        df_final=df2
    else:
        df_final=df_final.append(df2)
df_final=df_final.rename(columns={'mas_de1km':'MasDe1km','_mas_de1km':'%MasDe1km','mediana1_km':'Mediana1KM','mes':'Mes','dia':'Dia'})
df_final_fs1 = df_final
tiempo = (time.time()-time_fecha)/60
print((time.time()-start_time)/60)


2.6818121989568073


#### Control adm4

In [24]:
df_final = []
start_time = time.time()
for i,fecha in enumerate(fechas):
    time_fecha = time.time()
    day = str(fecha.day)
    if len(day) == 1:
        day = f'0{day}'
    codigo_fecha = str(fecha.month) + day
    #print(codigo_fecha)
    tabla_usuarios_fecha = f'historico_latlong_{pais}_{codigo_fecha}_nivel4_fs'
    query = f'''SELECT * FROM {tabla_usuarios_fecha}'''
    print(f"{codigo_fecha}")
    df2 = cursor.execute(query).as_pandas()
    if (codigo_fecha=='201'):
        df_final=df2
    else:
        df_final=df_final.append(df2)
df_final=df_final.rename(columns={'mas_de1km':'MasDe1km','_mas_de1km':'%MasDe1km','mediana1_km':'Mediana1KM','mes':'Mes','dia':'Dia'})
df_final_fs4 = df_final
tiempo = (time.time()-time_fecha)/60
print((time.time()-start_time)/60)

201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
3.0294425964355467


In [25]:
## control nivel adm4
base_control=df_final_fs1
base_control_v4=df_final_fs4
base_control_v4["adm1"]=""
base_control_v4.loc[base_control_v4["adm4"].astype(str).str.len()==7,["adm1"]]=base_control_v4["adm4"][base_control_v4["adm4"].astype(str).str.len()==7].map(lambda x: str(x)[0:2])
base_control4=base_control_v4.groupby(['Mes','Dia','adm1']).agg({'MasDe1km':'sum','n_users':'sum','%MasDe1km':'mean','Mediana1KM':'median'}).reset_index()
base_control4['adm1']=base_control4['adm1'].astype('int')
base_control4['adm1_v1']=""

for ids in list(cod_region['id_ibge'].astype('int')):
    base_control4.loc[base_control4['adm1']== ids , ['adm1_v1']] =  int(cod_region.loc[cod_region['id_ibge']== ids, 'id_provincia'])

base_control4 =base_control4.loc[:,['Mes','Dia','adm1_v1','MasDe1km','n_users','%MasDe1km','Mediana1KM']]
base_control4 =base_control4.rename(columns={'adm1_v1':'adm1','MasDe1km':'MasDe1km_adm4','n_users':'n_users_adm4','%MasDe1km':'%MasDe1km_adm4','Mediana1KM':'Mediana1KM_adm4'})

base_contro4=base_control.merge(base_control4,how='left',on=['Mes','Dia','adm1'])
base_contro4['%MasDe1km_adm4']=base_contro4['MasDe1km_adm4']/base_contro4['n_users_adm4']*100

cora=base_contro4.groupby('adm1')[['MasDe1km','MasDe1km_adm4']].corr().unstack().iloc[:,1].min()
corb=base_contro4.groupby('adm1')[['n_users','n_users_adm4']].corr().unstack().iloc[:,1].min()
corc=base_contro4.groupby('adm1')[['%MasDe1km','%MasDe1km_adm4']].corr().unstack().iloc[:,1].min()

if (min(cora,corb,corc)<0.9): print("OJO") 
else: print(min(cora,corb,corc), "CONTINUE")

0.999931858704129 CONTINUE


#### Guardar archivos

In [26]:
nivel_4 = 'adm4'
ruta=f'graphdata-mobility-public/Results_niveles/{pais}/base_usuario/{pais}_{nivel_4}_baseuser.csv'
write_dataframe_to_csv_on_s3(df_final_fs4, 'iadbprod-csd-hub-analyticaldata', ruta)

Writing 497937 records to graphdata-mobility-public/Results_niveles/BR/base_usuario/BR_cd_geocmi_baseuser.csv
S3
(497937, 9)
